# retrieve run data from mongo

In [15]:
from pymongo import MongoClient
import polars as pl

client = MongoClient("localhost", 27017)
db = client["trader"]
documents = list(db["message"].find({}))
run_data = pl.DataFrame(documents)

filtered_data = run_data.filter(pl.col("trading_session_id") == run_data['trading_session_id'][-1])

preprocessed_data = [{
    'message_type': row['content']['type'],
    'order_book': row['content'].get('order_book'),
    'incoming': {k: v for k, v in row['content'].get('incoming_message', {}).items() if k != 'trader_id'},
    # 'trader_id': row['content']['incoming_message'].get('trader_id'),
    
    'timestamp': row['timestamp']
} for row in filtered_data.to_dicts()]

new_order_books = pl.DataFrame(preprocessed_data).sort("timestamp", nulls_last=True).with_row_index("row_number")
print(new_order_books.head(10))

shape: (10, 5)
┌────────────┬──────────────┬─────────────────────────┬──────────────────┬─────────────────────────┐
│ row_number ┆ message_type ┆ order_book              ┆ incoming         ┆ timestamp               │
│ ---        ┆ ---          ┆ ---                     ┆ ---              ┆ ---                     │
│ u32        ┆ str          ┆ struct[2]               ┆ struct[3]        ┆ datetime[μs]            │
╞════════════╪══════════════╪═════════════════════════╪══════════════════╪═════════════════════════╡
│ 0          ┆ ADD_ORDER    ┆ {[],[{2001.0,1.0}]}     ┆ {1,2001.0,-1}    ┆ 2024-06-10 15:03:59.904 │
│ 1          ┆ ADD_ORDER    ┆ {[{1999.0,1.0}],[{2001. ┆ {1,1999.0,1}     ┆ 2024-06-10 15:03:59.970 │
│            ┆              ┆ 0,1.0}]}                ┆                  ┆                         │
│ 2          ┆ ADD_ORDER    ┆ {[{1999.0,1.0}],[{2001. ┆ {1,2001.0,-1}    ┆ 2024-06-10 15:04:00.040 │
│            ┆              ┆ 0,2.0}]}                ┆                  ┆  

In [16]:
new_order_books.head(10)

row_number,message_type,order_book,incoming,timestamp
u32,str,struct[2],struct[3],datetime[μs]
0,"""ADD_ORDER""","{[],[{2001.0,1.0}]}","{1,2001.0,-1}",2024-06-10 15:03:59.904
1,"""ADD_ORDER""","{[{1999.0,1.0}],[{2001.0,1.0}]}","{1,1999.0,1}",2024-06-10 15:03:59.970
2,"""ADD_ORDER""","{[{1999.0,1.0}],[{2001.0,2.0}]}","{1,2001.0,-1}",2024-06-10 15:04:00.040
3,"""BOOK_UPDATED""","{[{1999.0,1.0}],[{2001.0,2.0}]}","{null,null,null}",2024-06-10 15:04:00.079
4,"""ADD_ORDER""","{[{1999.0,2.0}],[{2001.0,2.0}]}","{1,1999.0,1}",2024-06-10 15:04:00.143
5,"""ADD_ORDER""","{[{1999.0,2.0}, {1997.0,1.0}],[{2001.0,2.0}]}","{1,1997.0,1}",2024-06-10 15:04:00.209
6,"""ADD_ORDER""","{[{1999.0,2.0}, {1997.0,1.0}],[{2000.0,1.0}, {2001.0,2.0}]}","{1,2000.0,-1}",2024-06-10 15:04:04.229
7,"""ADD_ORDER""","{[{1999.0,2.0}, {1997.0,1.0}],[{2001.0,2.0}]}","{1,2000.0,1}",2024-06-10 15:04:04.483
8,"""ADD_ORDER""","{[{1999.0,2.0}, {1997.0,1.0}, {1996.0,1.0}],[{2001.0,2.0}]}","{1,1996.0,1}",2024-06-10 15:04:06.808


In [155]:
filtered_data['content'][0]

{'content': 'Market is open',
 'type': 'BOOK_UPDATED',
 'order_book': {'bids': [], 'asks': []},
 'active_orders': [],
 'history': [],
 'spread': None,
 'midpoint': None,
 'transaction_price': None,
 'incoming_message': {'amount': None,
  'price': None,
  'order_type': None,
  'trader_id': None},
 'text': None}

In [153]:
filtered_data.head(10)

_id,trading_session_id,content,timestamp
object,str,struct[10],datetime[μs]
665e538d235236aa377dbe6d,"""8f3c43ba-e0be-…","{""Market is open"",""BOOK_UPDATED"",{[],[]},[],[],null,null,null,{null,null,null,null},null}",2024-06-04 00:36:45.097
665e538d235236aa377dbe6e,"""8f3c43ba-e0be-…","{null,""ADD_ORDER"",{[{2000.0,1.0}],[]},[{b""\xdb\x84\xac0\xee(Ag\xae\xdd_:\xd5\x09D\xde"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",1,1.0,2000.0,2024-06-03 23:36:45.132}],[],null,null,null,{1,2000.0,1,""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59""},""add_order update processed""}",2024-06-04 00:36:45.186
665e538e235236aa377dbe6f,"""8f3c43ba-e0be-…","{null,""ADD_ORDER"",{[],[]},[],[],null,null,null,{1,2000.0,-1,""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59""},""add_order update processed""}",2024-06-04 00:36:46.973
665e538f235236aa377dbe70,"""8f3c43ba-e0be-…","{null,""ADD_ORDER"",{[{2001.0,1.0}],[]},[{b""\xbdF\xc0\xff\xafSL\x9c\x8a\xf6\xab\x85{5\x8aF"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",1,1.0,2001.0,2024-06-03 23:36:47.065}],[{""a48c6e4f-47a7-432f-a73d-408658a3da9f"",""8f3c43ba-e0be-475d-b3e7-1cd5ad99012e"",""db84ac30-ee28-4167-aedd-5f3ad50944de"",""f978b8b1-3979-4e1e-91f2-40f6b0e25900"",2024-06-04 00:36:46.571,2000.0}],null,null,2000.0,{1,2001.0,1,""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59""},""add_order update processed""}",2024-06-04 00:36:47.154
665e5391235236aa377dbe71,"""8f3c43ba-e0be-…","{null,""ADD_ORDER"",{[{2001.0,1.0}],[{2011.0,1.0}]},[{b""\xbdF\xc0\xff\xafSL\x9c\x8a\xf6\xab\x85{5\x8aF"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",1,1.0,2001.0,2024-06-03 23:36:47.065}, {b""K\xdcGo\xf6\xebKs\x9a4\x9e\xa3\xae\xc6\xc5\x9f"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",-1,1.0,2011.0,2024-06-03 23:36:49.317}],[{""a48c6e4f-47a7-432f-a73d-408658a3da9f"",""8f3c43ba-e0be-475d-b3e7-1cd5ad99012e"",""db84ac30-ee28-4167-aedd-5f3ad50944de"",""f978b8b1-3979-4e1e-91f2-40f6b0e25900"",2024-06-04 00:36:46.571,2000.0}],10.0,2006.0,2000.0,{1,2011.0,-1,""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59""},""add_order update processed""}",2024-06-04 00:36:49.452
665e5391235236aa377dbe72,"""8f3c43ba-e0be-…","{null,""CANCEL_ORDER"",{[],[{2011.0,1.0}]},[{b""K\xdcGo\xf6\xebKs\x9a4\x9e\xa3\xae\xc6\xc5\x9f"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",-1,1.0,2011.0,2024-06-03 23:36:49.317}],[{""a48c6e4f-47a7-432f-a73d-408658a3da9f"",""8f3c43ba-e0be-475d-b3e7-1cd5ad99012e"",""db84ac30-ee28-4167-aedd-5f3ad50944de"",""f978b8b1-3979-4e1e-91f2-40f6b0e25900"",2024-06-04 00:36:46.571,2000.0}],null,null,2000.0,{-1,2001.0,1,""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59""},""cancel_order update processed""}",2024-06-04 00:36:49.550
665e5391235236aa377dbe73,"""8f3c43ba-e0be-…","{null,""ADD_ORDER"",{[{2000.0,1.0}],[{2011.0,1.0}]},[{b""K\xdcGo\xf6\xebKs\x9a4\x9e\xa3\xae\xc6\xc5\x9f"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",-1,1.0,2011.0,2024-06-03 23:36:49.317}, {b""\xcd\x18X\xb9\xe6\x1fO\xfe\xb7a\xf2\xe6T}T\xc2"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",1,1.0,2000.0,2024-06-03 23:36:49.800}],[{""a48c6e4f-47a7-432f-a73d-408658a3da9f"",""8f3c43ba-e0be-475d-b3e7-1cd5ad99012e"",""db84ac30-ee28-4167-aedd-5f3ad50944de"",""f978b8b1-3979-4e1e-91f2-40f6b0e25900"",2024-06-04 00:36:46.571,2000.0}],11.0,2005.5,2000.0,{1,2000.0,1,""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59""},""add_order update processed""}",2024-06-04 00:36:49.920
665e5392235236aa377dbe74,"""8f3c43ba-e0be-…","{null,""CANCEL_ORDER"",{[{2000.0,1.0}],[]},[{b""\xcd\x18X\xb9\xe6\x1fO\xfe\xb7a\xf2\xe6T}T\xc2"",""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59"",1,1.0,2000.0,2024-06-03 23:36:49.800}],[{""a48c6e4f-47a7-432f-a73d-408658a3da9f"",""8f3c43ba-e0be-475d-b3e7-1cd5ad99012e"",""db84ac30-ee28-4167-aedd-5f3ad50944de"",""f978b8b1-3979-4e1e-91f2-40f6b0e25900"",2024-06-04 00:36:46.571,2000.0}],null,null,2000.0,{-1,2011.0,-1,""NOISE_b776e3b3-f0ec-439e-b47a-91518a59ee59""},""cancel_order update processed""}",2024-06-04 00:36:50.008
665e5392235236aa377dbe75,"""8f3c43ba-e0be-…","{null,""ADD_ORDER"",{[

In [27]:
new_order_books.head(10)

row_number,order_book,incoming,timestamp
u32,struct[2],struct[4],datetime[μs]
0,"{[],[]}","{null,null,null,null}",2024-06-03 00:48:40.647
1,"{[{2000.0,1.0}],[]}","{1,2000,1,null}",2024-06-03 00:48:40.766
2,"{[],[]}","{1,2000,-1,null}",2024-06-03 00:48:40.867
3,"{[{2001.0,1.0}],[]}","{1,2001,1,null}",2024-06-03 00:48:40.955
4,"{[{2001.0,1.0}],[{2011.0,1.0}]}","{1,2011,-1,null}",2024-06-03 00:48:41.048
5,"{[{2001.0,1.0}, {2000.0,1.0}],[{2011.0,1.0}]}","{1,2000,1,null}",2024-06-03 00:48:41.154
6,"{[{2000.0,1.0}],[{2011.0,1.0}]}","{1,2000,-1,null}",2024-06-03 00:48:41.261
7,"{[{2000.0,2.0}],[{2011.0,1.0}]}","{1,2000,1,null}",2024-06-03 00:48:41.346
8,"{[{2000.0,1.0}],[{2011.0,1.0}]}","{1,2000,-1,null}",2024-06-03 00:48:41.488


In [28]:
from analysis.utilities import delete_all_tables

delete_all_tables()

DuckDB tables deleted successfully.
MongoDB collection deleted successfully.
